### Critical Sections: The Synchronization Problem

We are now going to take a deeper look inside synchronization of threads/processes and the principles needed to share data safely. There are two typical goals:
* Contention: 
  * How to resolve the conflicts that result from multiple processes trying to access shared resources?
* Cooperation:
  * An action by one process may enable another action by another process
  * In such cases, processes should coordinate their actions
